In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('/Users/riadanas/Desktop/housing regression MLE/data/raw/untouched_raw_original.csv')

In [7]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows

In [8]:
df.shape

(884092, 39)

In [9]:
df.head(2)

date  median_sale_price  median_list_price  median_ppsf  \
0  2012-03-31            46550.0           217450.0    31.813674   
1  2012-04-30            61870.0           245000.0    40.723982   

   median_list_ppsf  homes_sold  pending_sales  new_listings  inventory  \
0        110.183666        14.0           23.0          44.0       64.0   
1        130.528256        22.0           29.0          56.0       69.0   

   median_dom  avg_sale_to_list  sold_above_list  off_market_in_two_weeks  \
0        59.5          0.943662         0.142857                 0.043478   
1        89.5          0.946642         0.090909                 0.034483   

  city  zipcode  year  bank  bus  hospital  mall  park  restaurant  school  \
0  ATL    30002  2012  12.0  2.0       4.0   1.0  60.0        45.0    57.0   
1  ATL    30002  2012  12.0  2.0       4.0   1.0  60.0        45.0    57.0   

   station  supermarket  Total Population  Median Age  Per Capita Income  \
0      4.0          7.0            5811.0        36.3            33052.0   
1      4.0          7.0            5811.0        36.3            33052.0   

   Total Families Below Poverty  Total Housing Units  Median Rent  \
0                        5811.0               2677.0        710.0   
1                        5811.0               2677.0        710.0   

   Median Home Value  Total Labor Force  Unemployed Population  \
0           279500.0             3171.0                  460.0   
1           279500.0             3171.0                  460.0   

   Total School Age Population  Total School Enrollment  Median Commute Time  \
0                       5408.0                   5408.0               2492.0   
1                       5408.0                   5408.0               2492.0   

           price                         city_full  
0  200773.999557  Atlanta-Sandy Springs-Alpharetta  
1  202421.064584  Atlanta-Sandy Springs-Alpharetta

In [10]:
df['date'] = pd.to_datetime(df['date'])
print(df['date'].dt.year.value_counts())

date
2013    74712
2014    74712
2015    74712
2016    74712
2017    74712
2018    74712
2019    74712
2020    74712
2021    74712
2022    74712
2023    74712
2012    62260
Name: count, dtype: int64


##### Avoid Leakage

- Your train set (2012–2019) is “clean” historical training.

- Your eval set (2020–2021) is “unseen” during training, but still used for validation and hyperparameter tuning.

- Your holdout set (2022–2023) is completely untouched until the very end.

In [11]:
# Sort by date (important to avoid shuffling issues)
df = df.sort_values("date")

In [12]:
# Define cutoff dates
cutoff_date_eval = "2020-01-01"   # validation starts
cutoff_date_holdout = "2022-01-01"  # holdout starts

In [13]:
# Train: before 2020
train_df = df[df["date"] < cutoff_date_eval]

# Validation/Eval: 2020–2021
eval_df = df[(df["date"] >= cutoff_date_eval) & (df["date"] < cutoff_date_holdout)]

# Holdout: 2022–2023
holdout_df = df[df["date"] >= cutoff_date_holdout]

print("Train shape:", train_df.shape)
print("Eval shape:", eval_df.shape)
print("Holdout shape:", holdout_df.shape)

Train shape: (585244, 39)
Eval shape: (149424, 39)
Holdout shape: (149424, 39)


In [14]:
train_df.head(2)

date  median_sale_price  median_list_price  median_ppsf  \
0      2012-03-31            46550.0           217450.0    31.813674   
640562 2012-03-31           200000.0             7500.0   104.931794   

        median_list_ppsf  homes_sold  pending_sales  new_listings  inventory  \
0             110.183666        14.0           23.0          44.0       64.0   
640562         79.265873         1.0            1.0           1.0        2.0   

        median_dom  avg_sale_to_list  sold_above_list  \
0             59.5          0.943662         0.142857   
640562       290.0          0.909091         0.000000   

        off_market_in_two_weeks city  zipcode  year  bank  bus  hospital  \
0                      0.043478  ATL    30002  2012  12.0  2.0       4.0   
640562                 0.000000  PGH    15469  2012   0.0  0.0       0.0   

        mall  park  restaurant  school  station  supermarket  \
0        1.0  60.0        45.0    57.0      4.0          7.0   
640562   0.0   0.0         0.0     4.0      0.0          0.0   

        Total Population  Median Age  Per Capita Income  \
0                 5811.0        36.3            33052.0   
640562            2441.0        41.8            20241.0   

        Total Families Below Poverty  Total Housing Units  Median Rent  \
0                             5811.0               2677.0        710.0   
640562                        2385.0               1108.0        641.0   

        Median Home Value  Total Labor Force  Unemployed Population  \
0                279500.0             3171.0                  460.0   
640562            94600.0             1171.0                   52.0   

        Total School Age Population  Total School Enrollment  \
0                            5408.0                   5408.0   
640562                       2376.0                   2376.0   

        Median Commute Time          price                         city_full  
0                    2492.0  200773.999557  Atlanta-Sandy Springs-Alpharetta  
640562               1018.0  105863.681174                        Pittsburgh

In [15]:
eval_df.head(2)

date  median_sale_price  median_list_price  median_ppsf  \
137976 2020-01-31           184000.0           168450.0   123.333333   
157714 2020-01-31           140000.0           139000.0    94.691957   

        median_list_ppsf  homes_sold  pending_sales  new_listings  inventory  \
137976        109.899800        35.0           30.0          34.0       37.0   
157714        111.311054        61.0           81.0          83.0       54.0   

        median_dom  avg_sale_to_list  sold_above_list  \
137976        50.0          0.954732         0.171429   
157714        10.0          0.963039         0.098361   

        off_market_in_two_weeks city  zipcode  year  bank  bus  hospital  \
137976                 0.300000  CHI    60482  2020   3.0  0.0       2.0   
157714                 0.469136  CIN    41071  2020  35.0  5.0       4.0   

        mall   park  restaurant  school  station  supermarket  \
137976   0.0   68.0        30.0    60.0      5.0         14.0   
157714   3.0  139.0       259.0    86.0      1.0          8.0   

        Total Population  Median Age  Per Capita Income  \
137976           10849.0        42.1            27037.0   
157714           20624.0        36.6            30324.0   

        Total Families Below Poverty  Total Housing Units  Median Rent  \
137976                       10795.0               4598.0        964.0   
157714                       19423.0              10164.0        802.0   

        Median Home Value  Total Labor Force  Unemployed Population  \
137976           169000.0             5690.0                  414.0   
157714           132400.0            10479.0                  552.0   

        Total School Age Population  Total School Enrollment  \
137976                      10573.0                  10573.0   
157714                      19809.0                  19809.0   

        Median Commute Time          price                 city_full  
137976               5089.0  164839.734849  Chicago-Naperville-Elgin  
157714               9383.0  132164.314962                Cincinnati

In [16]:
holdout_df.head(2)

date  median_sale_price  median_list_price  median_ppsf  \
587856 2022-01-31           340000.0           349999.0   189.867865   
878956 2022-01-31           310000.0           269900.0   243.839836   

        median_list_ppsf  homes_sold  pending_sales  new_listings  inventory  \
587856        195.526649       282.0          287.0         241.0       73.0   
878956        239.543726        49.0           47.0          43.0        7.0   

        median_dom  avg_sale_to_list  sold_above_list  \
587856        12.0          0.991177         0.308511   
878956         6.5          1.006586         0.489796   

        off_market_in_two_weeks city  zipcode  year  bank  bus  hospital  \
587856                 0.634146  ORL    32771  2022   9.0  0.0       1.0   
878956                 0.723404  TPA    33778  2022  21.0  0.0       3.0   

        mall  park  restaurant  school  station  supermarket  \
587856   1.0  56.0        64.0    18.0      4.0          9.0   
878956   2.0  35.0        57.0    32.0      0.0         22.0   

        Total Population  Median Age  Per Capita Income  \
587856           57074.0        37.7            33896.0   
878956           15912.0        47.8            31277.0   

        Total Families Below Poverty  Total Housing Units  Median Rent  \
587856                       56681.0              22246.0       1289.0   
878956                       15405.0               7865.0       1161.0   

        Median Home Value  Total Labor Force  Unemployed Population  \
587856           261800.0            29485.0                 1469.0   
878956           206600.0             7846.0                  345.0   

        Total School Age Population  Total School Enrollment  \
587856                      55621.0                  55621.0   
878956                      15423.0                  15423.0   

        Median Commute Time          price                        city_full  
587856              24381.0  314667.746005        Orlando-Kissimmee-Sanford  
878956               6733.0  325271.243813  Tampa-St. Petersburg-Clearwater

In [17]:
# Save splits
train_df.to_csv("/Users/riadanas/Desktop/housing regression MLE/data/raw/train.csv", index=False)
eval_df.to_csv("/Users/riadanas/Desktop/housing regression MLE/data/raw/eval.csv", index=False)
holdout_df.to_csv("/Users/riadanas/Desktop/housing regression MLE/data/raw/holdout.csv", index=False)